### Imports

In [105]:
import torch.nn as nn
import torch as torch
import pandas as pd
import cv2
from tqdm import tqdm
import os


### Model Class Definition and Utility Functions

In [97]:
# Softmax image classifier class
class SoftmaxClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SoftmaxClassifier, self).__init__()
        
        # Single fully connected layer
        self.linear = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        
        # Compute raw logits
        logits = self.linear(x)
        
        # Apply softmax to get probabilities
        probabilities = nn.softmax(logits, dim=1)
        return probabilities

# Function to get the input dimension of a colored image
def get_input_dim(image_path):
    img_array = cv2.imread(image_path)
    flattened_array = img_array.reshape(-1)
    return len(flattened_array)

# Function to flatten a colored image into a 1D array
def flatten_image(image_path):
    img_array = cv2.imread(image_path)
    flattened_array = img_array.reshape(-1)
    return flattened_array

# Count total files in directory and subdirectories
def count_files(directory):
    total = 0

    for root, dirs, files in os.walk(directory):
        total += len(files)
    return total

### Configuration

In [98]:
# Model configuration
init_image_path = "/Users/ericcui/repos/imagenette/imagenette2-320/train/n01440764/ILSVRC2012_val_00000293.JPEG"
num_classes = 10
num_epochs = 10
base_learning_rate = 0.001
batch_size = 32

# CSV processing configuration
train_image_label_csv = "/Users/ericcui/repos/imagenette/imagenette2-320/train_imagenette.csv"
path_prefix = "/Users/ericcui/repos/imagenette/imagenette2-320/"

### Prototyping

In [94]:
# Initialize the softmax classifier
input_dim = get_input_dim(init_image_path)
softmax_classifier = SoftmaxClassifier(input_dim, num_classes)

# Initialize the optimizer
optimizer = torch.optim.Adam(softmax_classifier.parameters(), lr=base_learning_rate)

# Read the CSV file containing image paths and labels
df = pd.read_csv(train_image_label_csv)

In [124]:
# Function to convert images from image paths to flattened tensors in a batch dataframe
def process_batch_images(batch_df):
    images = batch_df['path'].tolist()
    images = [path_prefix + path for path in images]
    images = [flatten_image(path) for path in images]
    
    images = [torch.tensor(img, dtype=torch.float32) for img in images]
    return images

In [ ]:
batch_df = df.iloc[0:32]
images = process_batch_images(batch_df)
labels = batch_df['noisy_labels_0'].tolist()

# Forward pass
outputs = softmax_classifier(torch.stack(images))
loss = F.cross_entropy(outputs, torch.tensor(labels))

# Backward pass and optimization
optimizer.zero_grad()
loss.backward()
optimizer.step()

print(f"Loss: {loss.item():.4f}")



In [125]:
for epoch in range(num_epochs):
    for i in range(0, len(df), batch_size):
        
        # Getting the dataframe for the batch
        batch_df = df.iloc[i:i+batch_size]
        
        # Converting images to flattened tensors and labels to list
        images = process_batch_images(batch_df)
        labels = batch_df['noisy_labels_0'].tolist()
        
        
        

KeyboardInterrupt: 